In [ ]:
from pytube import YouTube, Search
import os

def progress_hook(stream, chunk, bytes_remaining):
    current = stream.filesize - bytes_remaining
    percent = round((current / stream.filesize) * 100, 2)
    print(f"Downloading: {percent}% complete")

query = "Pokemon GO Ultra League"
output_directory = "ultra_league"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

search = Search(query)
num_results = 10

for idx, video in enumerate(search.results[:num_results]):
    print(f"Downloading video {idx + 1} of {num_results}: {video.title}")
    yt = YouTube(video.watch_url)
    yt.register_on_progress_callback(progress_hook)
    yt_stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
    yt_stream.download(output_path=output_directory)
    print(f"Finished downloading {video.title}")


In [ ]:
import cv2
import os
import pytesseract

def save_annotated_frames(video_path, output_directory, roi_name, roi_annotation):
    cap = cv2.VideoCapture(video_path)

    start_from = 1200
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_from)

    frame_idx = start_from
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Draw ROI rectangles for debugging
        cv2.rectangle(frame, (roi_name[0], roi_name[1]), (roi_name[0] + roi_name[2], roi_name[1] + roi_name[3]), (0, 255, 0), 2)
        cv2.rectangle(frame, (roi_annotation[0], roi_annotation[1]), (roi_annotation[0] + roi_annotation[2], roi_annotation[1] + roi_annotation[3]), (0, 0, 255), 2)
        cv2.imshow('Frame with ROIs', frame)
        cv2.waitKey(1)

        # Get opponent Pokémon name from the name ROI
        roi_name_img = frame[roi_name[1]:roi_name[1] + roi_name[3], roi_name[0]:roi_name[0] + roi_name[2]]
        pokemon_name = pytesseract.image_to_string(roi_name_img).strip()
        print(pokemon_name)
        # Save frame with annotated Pokémon name in the annotation ROI
        if pokemon_name:
            cv2.putText(frame, pokemon_name, (roi_annotation[0], roi_annotation[1] + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            output_file = os.path.join(output_directory, f"frame_{frame_idx:04d}.jpg")
            cv2.imwrite(output_file, frame)

        frame_idx += 1

    cap.release()
    cv2.destroyAllWindows()


# Example usage:
video_path = 'ultra_league/1.mp4'
output_directory = 'output_frames'
roi_name = (380, 32, 30, 10)  # opponent Pokemon name ROI
roi_annotation = (150, 310, 50, 50)  # location of Pokemon (x,y,width, height)

save_annotated_frames(video_path, output_directory, roi_name, roi_annotation)
